<a href="https://colab.research.google.com/github/heyday1006/2D-Lidar-DNN/blob/main/dnn_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import os, re, math, json, shutil, pprint
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import IPython.display as display
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
print("Tensorflow version " + tf.__version__)

# Testing data

In [ ]:
import pickle5 as pickle
from numpy import linalg as LA
import math
from sklearn.utils import shuffle

def read_from_pickle(dir_name,world_name,size_change):
    print(os.path.exists(dir_name+'/'+world_name+'_range_pairs.pickle'))
    with open(dir_name+'/'+world_name+'_range_pairs.pickle', 'rb') as handle:
        range_pairs_all = pickle.load(handle)

    with open(dir_name+'/'+world_name+'_pose_rels.pickle', 'rb') as handle:
        pose_rels_all = pickle.load(handle)
    range_pairs = np.swapaxes(range_pairs_all, 1, 2)
    translation_rels=np.round(np.double(pose_rels_all[:,0:2]),4)
    rotation_rels=np.double(pose_rels_all[:,2])
    if size_change:
        range_pairs = range_pairs[:,::2,:]
        range_pairs = range_pairs[:,90:451,:]
    print("size of range_pairs: ", range_pairs.shape)
    print("size of pose_rels: ", translation_rels.shape)
    print("size of rotation_rels: ", rotation_rels.shape)
    print("<------------->")
    return range_pairs,translation_rels,rotation_rels

testing_range_pairs,testing_translation_rels,testing_rotation_rels = read_from_pickle('ros_data','custom_officetwo',True)
# testing_range_pairs,testing_translation_rels,testing_rotation_rels = read_from_pickle('benchmark_csail_0403','benchmark_csail',False)

In [ ]:
testing_pose_rels_all=np.hstack((testing_translation_rels,np.expand_dims(testing_rotation_rels, axis=1)))
#print(np.shape(range_pairs_compressed),np.shape(pose_rels_grid),np.shape(rotation_rels_grid),np.shape(pose_rels_grid_all),np.shape(pose_rels_all))
testing_range_pairs_all=np.clip(testing_range_pairs, a_min=None, a_max=40)    #remove the inf value of range to a large number
nonzero_indices = np.where((LA.norm(testing_pose_rels_all[:,0:2], axis=1)>1e-4) | (np.abs(testing_pose_rels_all[:,2])>1e-4))
x_test_new, y_test_new, y_test_new_ground_truth = (testing_range_pairs_all[nonzero_indices[0][0]:nonzero_indices[0][-1]],testing_pose_rels_all[nonzero_indices[0][0]:nonzero_indices[0][-1]], testing_pose_rels_all[nonzero_indices[0][0]:nonzero_indices[0][-1]])

x_pose_testing = np.array(x_test_new, dtype=float)
y_pose_testing = np.array(y_test_new, dtype=float)
print('Size of testing data: ', x_pose_testing.shape)
print('Size of testing labels: ',y_pose_testing.shape)

In [ ]:
nonzero_indices[0][-1]

# DL MODEL (Pose Regression)

In [ ]:
BATCH_SIZE = 32
pose_testing_dataset = tf.data.Dataset.from_tensor_slices((x_pose_testing, (y_pose_testing[:,0:2],y_pose_testing[:,2]))).batch(BATCH_SIZE)

In [ ]:
from tensorflow.keras.layers import Input, Conv1D, Activation, MaxPooling1D, Add, AveragePooling1D, Dense, Dropout, Flatten
from tensorflow.keras import Model

def residual_block(x):
        temp = x
        x = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(x)
        x = Conv1D(filters=64, kernel_size=3, padding='same')(x)
        x = Add()([Activation('relu')(x), temp])
        return x

def Res1DFlat_Pose(dropout_rate,input_shape):
    # input size = (batch_size, 2, 1081)
    input_data = Input(shape=input_shape)
    #x = Dense(1024, activation='relu')(1081)
    # first resnet block
    x = Conv1D(filters=64, kernel_size=7, strides=3, activation='relu', padding='same')(input_data)
    x = MaxPooling1D(pool_size=3)(x)
    x=residual_block(x)
    # second resnet block
    x=residual_block(x)
    # third resnet block
    x=residual_block(x)
    # fourth resnet block
    x=residual_block(x)
    # fifth resnet block
    x=residual_block(x)

    # Average pooling
    x = AveragePooling1D(pool_size=7)(x)
    x = Flatten()(x) # (batch_size, 2, channel)

    # Dense Layers
    x = Dense(1024, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(dropout_rate)(x)

    x1 = Dense(512, activation='relu')(x)
    x1 = Dropout(dropout_rate)(x1)
    #output1 = Dense(2786, activation='softmax', name='pose')(x1)
    output1 = Dense(2, activation=None, name='pose')(x1)

    x2 = Dense(512, activation='relu')(x)
    x2 = Dropout(dropout_rate)(x2)
    output2 = Dense(1, activation=None,name='rotation')(x2)
    #x2 = Dense(230, activation='softmax')(x2)
    #output2 = SoftArgmax(name='rotation')(x2,10)#Dense(230, activation='softmax', name='rotation')(x2)

    model = Model(inputs=input_data, outputs=[output1, output2])
    return model

In [ ]:
import zipfile
import gdown
# !gdown --id 1El4hPKw0Fn71kYD3Tj_fsdP3pVPjOUbI
# local_zip = 'model_weights_0404_361samples.zip'
# zip_ref = zipfile.ZipFile(local_zip, 'r')
# zip_ref.extractall('model_weights_0404_361samples')
# zip_ref.close()

In [ ]:
dropout_rate=0.2
pose_model = Res1DFlat_Pose(dropout_rate,np.shape(x_pose_testing[0]))
optimizer_lr=0.001
optimizer = tf.keras.optimizers.Adam(optimizer_lr)
pose_model.compile(optimizer=optimizer,
              loss={"pose": keras.losses.mse, "rotation": keras.losses.mse},#sparse_categorical_crossentropy,losses.mse},
              loss_weights=[10.0, 0.05],
              metrics={"pose": tf.keras.metrics.MeanAbsoluteError(), "rotation": tf.keras.metrics.MeanAbsoluteError()})#SparseCategoricalAccuracy,MeanAbsoluteError()})
# pose_model.summary()

In [ ]:
pose_model.load_weights('model_weights_0404_361samples/model_weights_0404_361samples/base_model_weights')
pose_model.evaluate(pose_testing_dataset)

In [ ]:
from matplotlib.animation import FuncAnimation
def edge_angle(yaw_j,yaw_jc):
    heading_j=np.array([[np.cos(yaw_j),np.sin(yaw_j)]]).T
    heading_jc=np.array([[np.cos(yaw_jc),np.sin(yaw_jc)]]).T
    cAngle=np.dot(heading_j.T,heading_jc)
    sAngle=np.dot(np.array([[0,0,1]]),np.cross(np.append(heading_j,0),np.append(heading_jc,0)))
    #print(np.shape(cAngle),np.shape(sAngle))
    edgeAngle=np.arctan2(sAngle,cAngle)
    return edgeAngle

def rels_to_abs(pose_rels,pose_init):
    pose_i = np.reshape(pose_init,(3,1))
    poses = pose_i.T
    theta_i = pose_init[2]
    for i in range(pose_rels.shape[0]):
        [rels_x,rels_y,rels_theta] = pose_rels[i,0:3]
        T_matrix = np.array([[np.cos(theta_i),-np.sin(theta_i)],[np.sin(theta_i),np.cos(theta_i)]])
        if isinstance(rels_theta,np.ndarray):
            theta_i += rels_theta[0,0]
        else:
            theta_i += rels_theta
        translation_ic = np.dot(T_matrix,np.reshape(list([rels_x,rels_y]),(2,1)))+pose_i[0:2]
        pose_ic =np.concatenate((translation_ic[0],translation_ic[1],[theta_i]),axis=0)[:,np.newaxis]
        poses = np.concatenate((poses,pose_ic.T))
        pose_i = pose_ic       
    return poses
trajectory_true = rels_to_abs(np.array(y_pose_testing),np.array([0,0,0]))#casil: [0.154,0.068,0.562729]))

In [ ]:
rels_to_abs(np.array(y_pose_testing[0:2]),np.array([0,0,0]))

In [ ]:
def posePredict(pose_model, x_pose_testing):
    pose_predict_raw=pose_model.predict(x_pose_testing)
    pose_predict_translation_value = np.array(pose_predict_raw[0])
#     print(np.array(tf.squeeze(pose_predict_raw[1])),np.array(tf.squeeze(pose_predict_raw[1])).shape)
    pose_predict_rotation_value=np.expand_dims(np.array(tf.squeeze(pose_predict_raw[1])), axis=1)*np.pi/180
    pose_predict = np.concatenate((pose_predict_translation_value,pose_predict_rotation_value),axis=-1)
    return np.array(pose_predict)
def display_posePredict(pose_model, x_pose_testing, y_pose_testing, nSamples,print_interval):
    pose_predict = posePredict(pose_model, x_pose_testing)
    for i in range(nSamples):
        predict_i=list(np.round(pose_predict[i,0:2],4))
        predict_i.append(np.round(float(pose_predict[i,2]),4))
        if i%print_interval==0:
            print("test: ", i, " ground_truth: ", np.round(np.double(y_pose_testing[i,:]),4),
              " predicted: ",predict_i)
    return pose_predict
pose_predict = display_posePredict(pose_model, x_pose_testing, y_pose_testing, nSamples=y_pose_testing.shape[0],print_interval=50)
trajectory_estimate = rels_to_abs(np.array(pose_predict),np.array([0,0,0]))#casil: [0.154,0.068,0.562729]))

In [ ]:
plt.figure(1)
plt.plot(range(x_pose_testing.shape[1]),x_pose_testing[0,:,0])
plt.plot(range(x_pose_testing.shape[1]),x_pose_testing[0,:,1])
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
def trajectory_plot(poses,ax):
#    plt.figure()
    colors = iter(cm.rainbow(np.linspace(1, 0, poses.shape[0] + 1)))
    for count in range(poses.shape[0]):
        if count % 2 == 0:
            ax.scatter(poses[count,0], poses[count,1], color=next(colors), s=35)
        if count == 9000:
            break
    ax.scatter(poses[0,0], poses[0,1], color='r', s=500)
    ax.scatter(poses[-1,0], poses[-1,1], color=next(colors), s=500)
    ax.plot(poses[:,0], poses[:,1])
plt.figure(1)
#    poses = poses[0:2,:]
ax1=plt.subplot(121)
trajectory_plot(trajectory_true[:,:2],ax1)
ax2=plt.subplot(122,sharex=ax1,sharey=ax1)
trajectory_plot(trajectory_estimate[:,:2],ax2)
plt.show()

In [ ]:
# with open('simple_maze5_pose_rels_estimate.pickle', 'wb') as handle:
#     pickle.dump(np.array(pose_predict), handle, protocol=pickle.HIGHEST_PROTOCOL)

# DL Model (Loop Closure)

In [ ]:
def Res1DFlat_Closure(dropout_rate,input_shape):
    # input size = (batch_size, 2, 1081)
    input_data = Input(shape=input_shape)
    #x = Dense(1024, activation='relu')(1081)
    # first resnet block
    x = Conv1D(filters=64, kernel_size=7, strides=3, activation='relu', padding='same')(input_data)
    x = MaxPooling1D(pool_size=3)(x)
    x=residual_block(x)
    # second resnet block
    x=residual_block(x)
    # third resnet block
    x=residual_block(x)
    # fourth resnet block
    x=residual_block(x)
    # fifth resnet block
    x=residual_block(x)

    # Average pooling
    x = AveragePooling1D(pool_size=7)(x)
    x = Flatten()(x) # (batch_size, 2, channel)

    # Dense Layers
    x = Dense(1024, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(dropout_rate)(x)

    x3 = Dense(512, activation='relu')(x)
    x3 = Dropout(dropout_rate)(x3)
    output3 = Dense(1, activation='sigmoid',name='closure')(x3)
    model = Model(inputs=input_data, outputs=[output3])
    return model

In [ ]:
# import zipfile
# import gdown
# !gdown --id 12Rz-4UxQLl-4riabQb7t_jw543ABfTYD
# local_zip = 'detection_model_weights_0404_361samples.zip'
# zip_ref = zipfile.ZipFile(local_zip, 'r')
# zip_ref.extractall('detection_model_weights_0404_361samples')
# zip_ref.close()

In [ ]:
detection_model = Res1DFlat_Closure(dropout_rate,np.shape(x_pose_testing[0]))
optimizer = tf.keras.optimizers.Adam(optimizer_lr)
detection_model.compile(optimizer=optimizer,
              loss={ "closure":keras.losses.BinaryCrossentropy()},#sparse_categorical_crossentropy,losses.mse},
              metrics={ "closure":[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.Precision(),tf.keras.metrics.Recall(),tf.keras.metrics.AUC()]})#SparseCategoricalAccuracy,MeanAbsoluteError()})
# detection_model.summary()
detection_model.load_weights('detection_model_weights_0411_361samples/base_model_weights')

In [ ]:
def closure_condtion_check(y_pose_testing):
    closure_indices=np.where((LA.norm(y_pose_testing[:,0:2], axis=1)<0.6) & (np.abs(y_pose_testing[:,2])<70*np.pi/180))
    return closure_indices

def closure_ground_truth_from_pose_rels(y_pose_testing):
    closure_indices = closure_condtion_check(y_pose_testing)
    closure_mask=np.zeros((y_pose_testing.shape[0],1),dtype=bool)
    closure_mask[closure_indices,:]=True
    y_closure_testing=closure_mask
    return y_closure_testing

def closure_pos_neg_rate(y_closure_testing):
    neg, pos = np.bincount(np.array(y_closure_testing[:,0]))
    total = neg + pos
    print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
        total, pos, 100 * pos / total))

def closurePredict(detection_model, x_closure_testing):
    closure_predict=detection_model.predict(x_closure_testing)>0.5
    return closure_predict

def display_closurePredidct(detection_model, x_closure_testing,y_closure_testing, nSamples,print_interval,display):
    BATCH_SIZE = 32
    detection_testing_dataset = tf.data.Dataset.from_tensor_slices((x_closure_testing, y_closure_testing)).batch(BATCH_SIZE)
    detection_model.evaluate(detection_testing_dataset)
    closure_predict = closurePredict(detection_model, x_closure_testing)
    if display:
        for i in range(nSamples):
            if i%print_interval==0:
                print("test: ", i, " ground_truth: ", y_closure_testing[i], " predicted: ",closure_predict[i])
    return closure_predict

In [ ]:
def closure_check(detection_model):
    testing_range_pairs,testing_translation_rels,testing_rotation_rels = read_from_pickle('ros_data','simple_maze_closure',True)
    testing_pose_rels_all=np.hstack((testing_translation_rels,np.expand_dims(testing_rotation_rels, axis=1)))
    #print(np.shape(range_pairs_compressed),np.shape(pose_rels_grid),np.shape(rotation_rels_grid),np.shape(pose_rels_grid_all),np.shape(pose_rels_all))
    testing_range_pairs_all=np.clip(testing_range_pairs, a_min=None, a_max=40)    #remove the inf value of range to a large number
    nonzero_indices = np.where((LA.norm(testing_pose_rels_all[:,0:2], axis=1)>1e-4) | (np.abs(testing_pose_rels_all[:,2])>1e-4))
    # nonfar_indices=np.where((testing_rotation_rels<=1.0) & (testing_rotation_rels>=-1.0) )
    # chosen_indices=np.intersect1d(nonzero_indices,nonfar_indices)
    chosen_indices = nonzero_indices
    x_test_new, y_test_new, y_test_new_ground_truth = (testing_range_pairs_all[chosen_indices],testing_pose_rels_all[chosen_indices], testing_pose_rels_all[chosen_indices])
    y_pose_testing = np.array(y_test_new, dtype=float)
    x_closure_testing = x_test_new
    y_closure_testing = closure_ground_truth_from_pose_rels(y_pose_testing)
    print(x_closure_testing.shape,y_closure_testing.shape)#,np.array(y_closure_testing[:,0]).shape)
    closure_pos_neg_rate(y_closure_testing)
    closure_predict = display_closurePredidct(detection_model, x_closure_testing,y_closure_testing, nSamples=y_closure_testing.shape[0], print_interval=500, display=False)
closure_check(detection_model)

# Pose Graph Optimization

In [ ]:
def abs_to_rels(pose_j,pose_jc):
    pose_delta=[]
    [x_j,y_j,yaw_j]=pose_j
    [x_jc,y_jc,yaw_jc]=pose_jc
    x_delta=(x_jc-x_j)*np.cos(yaw_j) + (y_jc-y_j)*np.sin(yaw_j)
    y_delta=(y_jc-y_j)*np.cos(yaw_j) - (x_jc-x_j)*np.sin(yaw_j)
    yaw_delta=edge_angle(yaw_j,yaw_jc)
    return [x_delta,y_delta,yaw_delta[0,0]]

# def trajectory_to_closure_candidates(trajectory_estimate):
#     # find out loop closure candidates
#     closure_candidates = []
#     for i in range(0,trajectory_estimate.shape[0]-15):
#         for j in range(i+15,trajectory_estimate.shape[0]):
#             pose_i = trajectory_estimate[i]
#             pose_j = trajectory_estimate[j]
#             pose_ij = abs_to_rels(pose_j,pose_i)
#     #         print(pose_ij[0:2])
#             if (LA.norm(pose_ij[0:2])<0.3) & (np.abs(pose_ij[2])<40*np.pi/180):
#                 closure_candidates.append([i,j])
#     closure_candidates = np.array(closure_candidates)
#     return closure_candidates


def closure_candidates_extensive(trajectory):
    closure_candidates = []
    for i in range(0, trajectory.shape[0]-20):
        for j in range(i+20, trajectory.shape[0]):
            closure_candidates.append([i,j])
    closure_candidates = np.array(closure_candidates)
    return closure_candidates

def pose_rels_from_trajectory(trajectory, closure_candidates):
    y_pose_testing = np.zeros((closure_candidates.shape[0],trajectory.shape[1]))
    for i in range(closure_candidates.shape[0]):
        y_pose_testing[i] = abs_to_rels(trajectory[closure_candidates[i,0]],trajectory[closure_candidates[i,1]])
    return y_pose_testing

def closure_ground_truth_from_trajectory(trajectory, closure_candidates):
    y_pose_testing = pose_rels_from_trajectory(trajectory, closure_candidates)
    closure_indices = closure_condtion_check(y_pose_testing)
    closure_mask=np.zeros((y_pose_testing.shape[0],1),dtype=bool)
    closure_mask[closure_indices,:]=True
    y_closure_testing=closure_mask
    return y_closure_testing          

def closure_candidates_scan_pairs(trajectory_estimate, testing_range_pairs, closure_candidates):
    # create loop closure candiates scan pairs
    laser_scans = np.zeros((trajectory_estimate.shape[0],testing_range_pairs.shape[1]))
    for i in range(trajectory_estimate.shape[0]-1):
        laser_scans[i] = testing_range_pairs[i,:,0]
    laser_scans[-1] = testing_range_pairs[-1,:,1]
    closure_range_pairs = []
    for i in range(closure_candidates.shape[0]):
        closure_range_pairs.append(laser_scans[closure_candidates[i],:])
    print(np.array(closure_range_pairs).shape)
    closure_range_pairs = np.swapaxes(np.array(closure_range_pairs), 1, 2)
    return closure_range_pairs

In [ ]:
def closure_candidates_filter(trajectory_estimate,trajectory_true, x_pose_testing):
    closure_candidates = closure_candidates_extensive(trajectory_estimate)
    closure_range_pairs = closure_candidates_scan_pairs(trajectory_estimate, x_pose_testing, closure_candidates)
    # print(closure_candidates.T, closure_range_pairs.shape)
    x_closure_testing = closure_range_pairs
    y_closure_testing = closure_ground_truth_from_trajectory(trajectory_true, closure_candidates)
    print("number of closure candidates using naive method: ", closure_candidates.shape[0])
#     closure_pos_neg_rate(y_closure_testing)
    # print(y_closure_testing)
    closure_predict = display_closurePredidct(detection_model, x_closure_testing,y_closure_testing, nSamples=x_closure_testing.shape[0], print_interval=100, display=False)
    # filter out outliers
    closure_candidates_with_outliers = closure_candidates[np.where(closure_predict==True)[0]]
    print("number of closures before removing outliers: ", closure_candidates_with_outliers.shape[0])
    pose_rels_odometry = pose_rels_from_trajectory(trajectory_estimate, closure_candidates_with_outliers)
    closure_indices = closure_condtion_check(pose_rels_odometry)
    closure_candidates_without_outliers = closure_candidates_with_outliers[closure_indices]
    closure_range_pairs = closure_candidates_scan_pairs(trajectory_estimate, x_pose_testing, closure_candidates_without_outliers)
    print("number of closures after removing outliers: ", closure_candidates_without_outliers.shape[0])
    x_closure_testing = closure_range_pairs
    y_closure_testing = closure_ground_truth_from_trajectory(trajectory_true, closure_candidates_without_outliers)
    closure_predict = display_closurePredidct(detection_model, x_closure_testing,y_closure_testing, nSamples=x_closure_testing.shape[0], print_interval=5, display=True)
    #from pose regressor find closure relative pose
    closure_pose_candidates = closure_candidates_without_outliers[:,[1, 0]]
    closure_pose_y_testing = pose_rels_from_trajectory(trajectory_true,closure_pose_candidates)
    closure_pose_x_testing = closure_candidates_scan_pairs(trajectory_estimate, x_pose_testing, closure_pose_candidates)
    closure_pose_predict = display_posePredict(pose_model, closure_pose_x_testing, closure_pose_y_testing, nSamples=closure_pose_y_testing.shape[0],print_interval=10)
    return closure_pose_candidates, closure_pose_predict

In [ ]:
closure_pose_candidates, closure_pose_predict = closure_candidates_filter(trajectory_estimate,trajectory_true, x_pose_testing)

In [ ]:
closure_pose_candidates

In [ ]:
closure_candidates_without_outliers = closure_pose_candidates
test_idx = closure_candidates_without_outliers[3]
test_range_sample=np.concatenate((x_pose_testing[test_idx[0],:,0][np.newaxis, :, np.newaxis],x_pose_testing[test_idx[1],:,0][np.newaxis, :, np.newaxis]),axis=-1)
test_odometry_sample=pose_rels_from_trajectory(trajectory_estimate, np.array([[test_idx[0],test_idx[1]]]))
closure_predict = closurePredict(detection_model, test_range_sample)
print(test_odometry_sample,closure_predict)
plt.figure(1)
#    poses = poses[0:2,:]
ax1=plt.subplot(121)
trajectory_plot(trajectory_true[:,:2],ax1)
for i in range(closure_candidates_without_outliers.shape[0]):
    test_idx = closure_candidates_without_outliers[i]
    ax1.scatter(trajectory_true[test_idx,0],trajectory_true[test_idx,1],color='g',s=20)
    
ax2=plt.subplot(122,sharex=ax1,sharey=ax1)
trajectory_plot(trajectory_estimate[:,:2],ax2)
for i in range(closure_candidates_without_outliers.shape[0]):
    test_idx = closure_candidates_without_outliers[i]
    ax2.scatter(trajectory_estimate[test_idx,0],trajectory_estimate[test_idx,1],color='g',s=20)
plt.show()

In [ ]:
import gtsam
import matplotlib.pyplot as plt
import gtsam.utils.plot as gtsam_plot
def vector3(x, y, z):
    """Create 3d double numpy array."""
    return np.array([x, y, z], dtype=float)

def pose_graph_construct(pose_predict,trajectory_estimate,closure_pose_candidates,closure_pose_predict):
    # Create noise models
    PRIOR_NOISE = gtsam.noiseModel.Diagonal.Sigmas(vector3(0.05, 0.05, 0.1))
    ODOMETRY_NOISE = gtsam.noiseModel.Diagonal.Sigmas(vector3(0.05, 0.05, 0.1))
    # 1. Create a factor graph container and add factors to it
    graph = gtsam.NonlinearFactorGraph()
    # 2b. Add odometry factors
    # A prior factor consists of a mean and a noise ODOMETRY_NOISE (covariance matrix)
    graph.add(gtsam.PriorFactorPose2(1, gtsam.Pose2(0, 0, 0), PRIOR_NOISE))
    # graph.add(gtsam.NonlinearEqualityPose2())
    # Create odometry (Between) factors between consecutive poses
    pose_predict=np.array(pose_predict)
    for i in range(pose_predict.shape[0]-1):
        pos_rel = pose_predict[i+1]
        graph.add(gtsam.BetweenFactorPose2(i+1, i+2, gtsam.Pose2(pos_rel[0],pos_rel[1], pos_rel[2]), ODOMETRY_NOISE))
    # print("\nFactor Graph:\n{}".format(graph))  # print
    # 2c. Add the loop closure constraint
    # This factor encodes the fact that we have returned to the same pose. In real
    # systems, these constraints may be identified in many ways, such as appearance-based
    # techniques with camera images. We will use another Between Factor to enforce this constraint:
    closure_pose_candidates_new = closure_pose_candidates + 1
    for i in range(closure_pose_candidates.shape[0]):
        graph.add(gtsam.BetweenFactorPose2(
            closure_pose_candidates_new[i,0], closure_pose_candidates_new[i,1], gtsam.Pose2(closure_pose_predict[i,0],closure_pose_predict[i,1],closure_pose_predict[i,2]), ODOMETRY_NOISE))
    # 3. Create the data structure to hold the initial_estimate estimate to the
    # solution. use odometry estimation as the initial estimate (nonconvex: initial matters)
    initial_estimate = gtsam.Values()
    for i in range(trajectory_estimate.shape[0]):
        initial_estimate.insert(i+1, gtsam.Pose2(trajectory_estimate[i,0], trajectory_estimate[i,1], trajectory_estimate[i,2]))
    # print("\nInitial Estimate:\n{}".format(initial_estimate))  # print
    return graph, initial_estimate

In [ ]:
def pose_graph_optimization(graph, initial_estimate):
    # 4. Optimize the initial values using a Gauss-Newton nonlinear optimizer
    # The optimizer accepts an optional set of configuration parameters,
    # controlling things like convergence criteria, the type of linear
    # system solver to use, and the amount of information displayed during
    # optimization. We will set a few parameters as a demonstration.
    parameters = gtsam.GaussNewtonParams()

    # Stop iterating once the change in error between steps is less than this value
    parameters.setRelativeErrorTol(1e-5)
    # Do not perform more than N iteration steps
    parameters.setMaxIterations(100)
    # Create the optimizer ... # other optimizers LM
    optimizer = gtsam.GaussNewtonOptimizer(graph, initial_estimate, parameters)
    # ... and optimize
    result = optimizer.optimize()
    # print("Final Result:\n{}".format(result))

    resultPoses = gtsam.utilities.extractPose2(result)
#     for i in range(resultPoses.shape[0]):
#         gtsam_plot.plot_pose2(1, gtsam.Pose2(resultPoses[i, :]))
#     plt.show()
    return resultPoses

In [ ]:
graph,initial_estimate = pose_graph_construct(pose_predict,trajectory_estimate,closure_pose_candidates,closure_pose_predict)
resultPoses = pose_graph_optimization(graph, initial_estimate)

In [ ]:
ax1=plt.subplot(131)
trajectory_plot(trajectory_true[:,:2],ax1)
plt.scatter(trajectory_true[test_idx,0],trajectory_true[test_idx,1],color='g',s=100)
ax2=plt.subplot(132)
trajectory_plot(trajectory_estimate[:,:2],ax2)
plt.scatter(trajectory_estimate[test_idx,0],trajectory_estimate[test_idx,1],color='g',s=100)
ax3=plt.subplot(133,sharex=ax1,sharey=ax1)
trajectory_plot(resultPoses[:,:2],ax3)
plt.scatter(resultPoses[test_idx,0],resultPoses[test_idx,1],color='g',s=100)
plt.show()